# Atividade

* Criar um banco de dados na ferramenta cloud (ScaleGrid) ou localmente
(PostgreSQL ou MySQL), chamado ecommerce, e definir as tabelas necessárias para o
sistema. Além disso, como desafio, você deve criar um script em Python para inserir 10 mil
registros em cada uma das tabelas criadas.

### Criando o Banco de Dados

In [ ]:
CREATE DATABASE ecommerce;

In [ ]:
CREATE TABLE categoria (
	id_categoria SERIAL NOT NULL,
	nome VARCHAR(255) NOT NULL,
	descricao TEXT NOT NULL,
	CONSTRAINT categoria_pkey PRIMARY KEY (id_categoria)
);

CREATE TABLE cliente (
	id_cliente SERIAL NOT NULL,
	nome VARCHAR(255) NOT NULL,
	endereco VARCHAR(255) NOT NULL,
	telefone VARCHAR(13) NOT NULL,
	email VARCHAR(255) NOT NULL,
	data_nascimento DATE NOT NULL,
	CONSTRAINT cliente_email_key UNIQUE (email),
	CONSTRAINT cliente_pkey PRIMARY KEY (id_cliente)
);

CREATE TABLE produto (
	id_produto SERIAL NOT NULL,
	nome VARCHAR(255) NOT NULL,
	descricao TEXT NOT NULL,
	preco NUMERIC(10, 2) NOT NULL,
	estoque INT NOT NULL,
	id_categoria INT NOT NULL,
	CONSTRAINT produto_pkey PRIMARY KEY (id_produto),
	CONSTRAINT fk_categoria FOREIGN KEY (id_categoria) REFERENCES categoria(id_categoria)
);

CREATE TABLE item_do_pedido (
	id_item_do_pedido SERIAL NOT NULL,
	quantidade INT NOT NULL,
	preco_unitario NUMERIC(10, 2) NOT NULL,
	subtotal NUMERIC(10, 2) NOT NULL,
	id_produto INT NOT NULL,
	CONSTRAINT item_do_pedido_pkey PRIMARY KEY (id_item_do_pedido),
	CONSTRAINT fk_produto FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);

CREATE TABLE pedido (
	id_pedido SERIAL NOT NULL,
	data_pedido DATE NOT NULL,
	status VARCHAR(50) NOT NULL,
	valor_total NUMERIC(10, 2) NOT NULL,
	id_item_do_pedido INT NOT NULL,
	id_cliente INT NOT NULL,
	CONSTRAINT pedido_pkey PRIMARY KEY (id_pedido),
	CONSTRAINT fk_cliente FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente),
	CONSTRAINT fk_item_do_pedido FOREIGN KEY (id_item_do_pedido) REFERENCES item_do_pedido(id_item_do_pedido)
);

CREATE TABLE pagamento (
	id_pagamento SERIAL NOT NULL,
	data_pagamento DATE NULL,
	valor_pago NUMERIC(10, 2) NULL,
	metodo_pagamento VARCHAR(50) NULL,
	id_pedido INT NOT NULL,
	CONSTRAINT pagamento_pkey PRIMARY KEY (id_pagamento),
	CONSTRAINT fk_pedido FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido)
);


### Script em Python para inserir os registros

In [2]:
import psycopg2
import random
from faker import Faker
from datetime import datetime, timedelta


In [12]:
fake = Faker()

# Realizando conexão local
conn = psycopg2.connect(
    dbname="ecommerce",
    user="user",
    password="password",
    host="host",
    port="port"
)

cur = conn.cursor()


In [4]:
# Função para gerar dados aleatórios com limite de comprimento
def truncate(text, max_length):
    return text[:max_length]


In [5]:
# Inserir registros na tabela 'categoria'
categoria_ids = []

for i in range(10000):
    nome = truncate(fake.word().capitalize(), 100)
    descricao = truncate(fake.sentence(), 255)

    cur.execute(
        "INSERT INTO categoria (nome, descricao) VALUES (%s, %s) RETURNING id_categoria",
        (nome, descricao)
    )

    categoria_ids.append(cur.fetchone()[0])
   

In [7]:
# Inserir registros na tabela 'produto'
produto_ids = []

for i in range(10000):
    nome = truncate(fake.word().capitalize(), 100)
    descricao = truncate(fake.sentence(), 255)
    preco = round(random.uniform(10.0, 100.0), 2)
    estoque = random.randint(1, 100)
    id_categoria = random.choice(categoria_ids)

    cur.execute(
        "INSERT INTO produto (nome, descricao, preco, estoque, id_categoria) VALUES (%s, %s, %s, %s, %s) RETURNING id_produto",
        (nome, descricao, preco, estoque, id_categoria)
    )
    
    produto_ids.append(cur.fetchone()[0])


In [13]:
# Inserir registros na tabela 'cliente'
cliente_ids = []

for i in range(10000):
    nome = truncate(fake.name(), 100)
    endereco = truncate(fake.address().replace('\n', ', '), 255)
    telefone = truncate(fake.phone_number(), 13)
    email = truncate(f"{nome.replace(' ', '').lower()}_{random.randint(1, 10000)}@example.com", 100)
    data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=90)

    cur.execute(
        "INSERT INTO cliente (nome, endereco, telefone, email, data_nascimento) VALUES (%s, %s, %s, %s, %s) RETURNING id_cliente",
        (nome, endereco, telefone, email, data_nascimento)
    )
    
    cliente_ids.append(cur.fetchone()[0])


In [15]:
# Inserir registros na tabela 'item_do_pedido'
item_do_pedido_ids = []

for i in range(10000):
    quantidade = random.randint(1, 10)
    preco_unitario = round(random.uniform(10.0, 100.0), 2)
    subtotal = quantidade * preco_unitario
    id_produto = random.choice(produto_ids)

    cur.execute(
        "INSERT INTO item_do_pedido (quantidade, preco_unitario, subtotal, id_produto) VALUES (%s, %s, %s, %s) RETURNING id_item_do_pedido",
        (quantidade, preco_unitario, subtotal, id_produto)
    )

    item_do_pedido_ids.append(cur.fetchone()[0])


In [17]:
# Inserir registros na tabela 'pedido'
pedido_ids = []

for i in range(10000):
    data_pedido = fake.date_between(start_date='-4y', end_date='today')
    status = truncate(random.choice(['Pendente', 'Concluído', 'Cancelado']), 20)
    valor_total = round(random.uniform(50.0, 500.0), 2)
    id_item_do_pedido = random.choice(item_do_pedido_ids)
    id_cliente = random.choice(cliente_ids)

    cur.execute(
        "INSERT INTO pedido (data_pedido, status, valor_total, id_item_do_pedido, id_cliente) VALUES (%s, %s, %s, %s, %s) RETURNING id_pedido" ,
        (data_pedido, status, valor_total, id_item_do_pedido, id_cliente)
    )

    pedido_ids.append(cur.fetchone()[0])


In [19]:
# Inserir registros na tabela 'pagamento'
for i in range(10000):
    data_pagamento = fake.date_between(start_date='-4y', end_date='today')
    valor_pago = round(random.uniform(50.0, 500.0), 2)
    metodo_pagamento = truncate(random.choice(['Cartão de Crédito', 'Boleto', 'Pix']), 50)
    id_pedido = random.choice(pedido_ids)

    cur.execute(
        "INSERT INTO pagamento (data_pagamento, valor_pago, metodo_pagamento, id_pedido) VALUES (%s, %s, %s, %s)",
        (data_pagamento, valor_pago, metodo_pagamento, id_pedido)
    )


In [21]:
# Fazer o commit e desfazer a conexão
conn.commit()
cur.close()
conn.close()
